In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

import time


In [2]:
# !pip install xgboost

In [3]:
from xgboost import XGBRanker

In [4]:
df = pd.read_csv('train_preprocessed_2.csv')
df.head()

,srch_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,...,month,day_of_week,relevance,prop_starrating_location_interaction,total_time_duration,highly_rated_prop,total_people,mean_price_by_country,has_promotion_and_highly_rated,normalized_price
0,1,219,893,3,3.5,1,2.83,0.0438,4.95,27,...,4,3,0.0,8.49,1,0,4,0.000941,0,0.997827
1,1,219,10404,4,4.0,1,2.20,0.0149,5.03,26,...,4,3,0.0,8.80,1,1,4,0.000428,0,0.996458
2,1,219,21315,3,4.5,1,2.20,0.0245,4.92,21,...,4,3,0.0,6.60,1,0,4,0.000616,0,0.996270
3,1,219,27348,2,4.0,1,2.83,0.0125,4.39,34,...,4,3,0.0,5.66,1,0,4,0.009392,0,0.987494
4,1,219,29604,4,3.5,1,2.64,0.1241,4.93,4,...,4,3,0.0,10.56,1,1,4,0.000136,0,0.997022


In [5]:
df.columns

Index(['srch_id', 'prop_country_id', 'prop_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_location_score2', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'click_bool', 'booking_bool', 'month', 'day_of_week', 'relevance',
       'prop_starrating_location_interaction', 'total_time_duration',
       'highly_rated_prop', 'total_people', 'mean_price_by_country',
       'has_promotion_and_highly_rated', 'normalized_price'],
      dtype='object')

In [6]:
X = df
train_groups = X.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

relevances = X['relevance']

X= X.loc[:, ~X.columns.isin(['srch_id','relevance'])]
y = relevances

In [7]:
X.columns

Index(['prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'random_bool', 'click_bool',
       'booking_bool', 'month', 'day_of_week',
       'prop_starrating_location_interaction', 'total_time_duration',
       'highly_rated_prop', 'total_people', 'mean_price_by_country',
       'has_promotion_and_highly_rated', 'normalized_price'],
      dtype='object')

In [8]:
X = X.drop(['booking_bool','position','random_bool','prop_id','click_bool'], axis=1)


In [ ]:
model = XGBRanker(
                  objective='rank:ndcg', 
                  n_estimators=500, 
                  random_state=0,
                  learning_rate=0.1)


model.fit(
    X,
    y,
    group=train_groups,
    eval_metric='ndcg@5',
    eval_set=[(X, y)],
    eval_group=[list(train_groups)],
    verbose=True,
    early_stopping_rounds=10
)

/Users/freekcool/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/freekcool/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-ndcg@5:0.97590
[1]	validation_0-ndcg@5:0.97673
[2]	validation_0-ndcg@5:0.97678
[3]	validation_0-ndcg@5:0.97698
[4]	validation_0-ndcg@5:0.97696
[5]	validation_0-ndcg@5:0.97694
[6]	validation_0-ndcg@5:0.97697
[7]	validation_0-ndcg@5:0.97703
[8]	validation_0-ndcg@5:0.97711
[9]	validation_0-ndcg@5:0.97711
[10]	validation_0-ndcg@5:0.97712
[11]	validation_0-ndcg@5:0.97712
[12]	validation_0-ndcg@5:0.97717
[13]	validation_0-ndcg@5:0.97720
[14]	validation_0-ndcg@5:0.97735
[15]	validation_0-ndcg@5:0.97739
[16]	validation_0-ndcg@5:0.97739
[17]	validation_0-ndcg@5:0.97743
[18]	validation_0-ndcg@5:0.97746
[19]	validation_0-ndcg@5:0.97749
[20]	validation_0-ndcg@5:0.97753
[21]	validation_0-ndcg@5:0.97767
[22]	validation_0-ndcg@5:0.97770
[23]	validation_0-ndcg@5:0.97778
[24]	validation_0-ndcg@5:0.97784
[25]	validation_0-ndcg@5:0.97799
[26]	validation_0-ndcg@5:0.97804
[27]	validation_0-ndcg@5:0.97807
[28]	validation_0-ndcg@5:0.97812
[29]	validation_0-ndcg@5:0.97820
[30]	validation_0-nd

[245]	validation_0-ndcg@5:0.98430
[246]	validation_0-ndcg@5:0.98432
[247]	validation_0-ndcg@5:0.98432
[248]	validation_0-ndcg@5:0.98433
[249]	validation_0-ndcg@5:0.98436
[250]	validation_0-ndcg@5:0.98436
[251]	validation_0-ndcg@5:0.98438
[252]	validation_0-ndcg@5:0.98440
[253]	validation_0-ndcg@5:0.98442
[254]	validation_0-ndcg@5:0.98445
[255]	validation_0-ndcg@5:0.98447
[256]	validation_0-ndcg@5:0.98449
[257]	validation_0-ndcg@5:0.98452
[258]	validation_0-ndcg@5:0.98456
[259]	validation_0-ndcg@5:0.98456
[260]	validation_0-ndcg@5:0.98460
[261]	validation_0-ndcg@5:0.98462
[262]	validation_0-ndcg@5:0.98464
[263]	validation_0-ndcg@5:0.98465
[264]	validation_0-ndcg@5:0.98466
[265]	validation_0-ndcg@5:0.98469
[266]	validation_0-ndcg@5:0.98471
[267]	validation_0-ndcg@5:0.98472
[268]	validation_0-ndcg@5:0.98473
[269]	validation_0-ndcg@5:0.98477
[270]	validation_0-ndcg@5:0.98479
[271]	validation_0-ndcg@5:0.98482
[272]	validation_0-ndcg@5:0.98484
[273]	validation_0-ndcg@5:0.98486
[274]	validati

[486]	validation_0-ndcg@5:0.98757
[487]	validation_0-ndcg@5:0.98759
[488]	validation_0-ndcg@5:0.98759
[489]	validation_0-ndcg@5:0.98761
[490]	validation_0-ndcg@5:0.98762
[491]	validation_0-ndcg@5:0.98764
[492]	validation_0-ndcg@5:0.98764


In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')

In [ ]:
test_df = pd.read_csv('test_preprocessed_2.csv')

In [ ]:
print(len(test_df))

In [ ]:
def sort_properties(srch_id, scores, prop_ids):
    sorted_group = pd.DataFrame(columns=['srch_id', 'prop_id', 'scores'])
    sorted_group['srch_id'] = len(scores)*[srch_id]
    sorted_group['prop_id'] = prop_ids.values
    sorted_group['scores'] = scores
    sorted_group = sorted_group.sort_values(by='scores', ascending=False)
    return sorted_group 

def recommend_properties(model, data):
    result = pd.DataFrame(columns=['srch_id', 'prop_id', 'scores'])
    df_list = []
    grouped_data = data.groupby('srch_id')
    for group_name, group in tqdm(grouped_data):
        scores = model.predict(group.loc[:, ~group.columns.isin(['srch_id','prop_id'])])
        sorted_group = sort_properties(group_name, scores, group['prop_id'])
        df_list.append(sorted_group)
    result = pd.concat(df_list)
    return result

In [ ]:
X.columns

In [ ]:
test_df.columns

In [ ]:
start = time.time()
recommendations = recommend_properties(model, test_df)
end = time.time()
print(f'Total Time for validation prediction:{end - start}')
submission = recommendations.drop(columns=['scores'])
submission.to_csv(f'xgbranker_predictions.csv', index=False)